In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline 
import random
import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llms-you-cant-please-them-all/sample_submission.csv
/kaggle/input/llms-you-cant-please-them-all/test.csv


In [5]:
# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

# Load test dataset
test_df = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/test.csv')
submission_df = pd.DataFrame(columns=["id", "essay"])
submission_df["id"] = test_df["id"]

Using device: GPU


In [6]:
# Initialize Language Model
# Load a better model for essay generation with GPU support
generator = pipeline("text-generation", model="gpt2", truncation=True, device=1)  # Uses GPU if available

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:1


In [7]:
# Define Essay Generation Function to generate diverse essays
def generate_diverse_essay(topic):
    """Generates an essay using diverse writing styles to maximize LLM disagreement."""
    prompts = [
        f"Write a formal academic essay on {topic}, including citations and structured arguments.",
        f"Write a satirical critique on {topic}, exaggerating contradictions and ironies.",
        f"Write a philosophical dialogue where two characters debate {topic} with opposing views.",
        f"Write a poem that metaphorically explores {topic} using vivid imagery.",
        f"Write a controversial argument about {topic}, making bold claims with limited evidence.",
    ]

    selected_prompt = random.choice(prompts)  # Randomly pick a style
    essay = generator(selected_prompt, max_length=150, truncation=True)[0]['generated_text']
    return essay

In [8]:
# Generate essays
test_df['essay'] = test_df['topic'].apply(generate_diverse_essay)
submission_df['essay'] = test_df['essay']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [9]:
print (submission_df['essay'].values)

['Write a satirical critique on Compare and contrast the importance of self-reliance and adaptability in healthcare., exaggerating contradictions and ironies. See, for example, P. V. Baskar, "The Politics of \'Self-Injunction\': Selfishness and Self-Possession" in D. A. Pyle, Studies of Health, pp. 2-4. In: "Herman Singer," American Journal of Sociology, p. 93., (1973).\n\n"Selfishness and Self-Injunction" (1978)\n\n"Barely a fortnight after the passing of the Republican bill, the federal government is preparing to put up a monument to the man who would become President of the'
 'Write a philosophical dialogue where two characters debate Evaluate the effectiveness of management consulting in addressing conflicts within marketing. with opposing views. I don\'t think you should try to go to business like this to help people to improve. The only better way is to focus on understanding the process itself and not trying too hard to get something out of it before you do. This helps people ge

In [11]:
# Save submission file
submission_df[["id", "essay"]].to_csv("submission.csv", index=False)

print("Submission file created successfully!")

Submission file created successfully!
